In [220]:
import uproot
import numpy as np
import os
import matplotlib.pyplot as plt

liczba = os.popen('cd traning/bkg; find -type f | wc -l').read()
liczba2 = int(liczba)

In [221]:
import glob
file_pattern = "traning/bkg/*.root"
files = glob.glob(file_pattern)

In [222]:
branch_list = [uproot.open(files[i])['nominal'].arrays(library="np") for i in range (liczba2) ]
signal = branch_list[-1]

In [223]:
combined_data = {}

In [224]:
def delete_branches(branches):
    del branches['jet_isbtagged_DL1r_77']
    del branches['mc_kFactor']
    del branches['RunYear']
    del branches['mc_rawXSection']
    del branches['lep_ID_0']
    del branches['p_Event_No']
    del branches['jet_pt']
    del branches['jet_eta']
    del branches['jet_phi']
    del branches['jet_e']
    del branches['lep_vz_0']
    del branches['lep_vz_1']
    del branches['taus_DL1r_0']
    del branches['p_sumElepAll_EJet']
    del branches['p_sumPtleptauAll_Ptjet']

In [225]:
for i in branch_list:
    delete_branches(i)


In [226]:
keys = list(branch_list[0])

In [227]:
def join_branchKey(arr):
    combo = arr
    if len(arr) > 1:
        combo = np.concatenate([arr[0], arr[1]])
        for i in range(len(arr)):
            combo = np.concatenate([combo, arr[i]])
    return combo

In [228]:
def join_branch(branch):
    for key in keys:
        if all(isinstance(item, np.ndarray) for item in branch[key]):
            branch[key] = join_branchKey(branch[key])
    return branch

In [229]:
for i in range(len(branch_list)):
    branch_list[i] = join_branch(branch_list[i])

In [230]:
combined_data

{}

In [231]:
for i in keys:
    combo = np.concatenate([branch_list[0][i], np.array([])])
    for k in branch_list:
        combo = np.concatenate([combo, k[i]])
    combined_data[i] = combo
    try:
        combined_data[i] = combined_data[i].astype(np.float32)
    except ValueError:
        print(i)

In [232]:
"""def join_signal_bkg(signal_branches, bkg_branches):
    branch_list_all = {}
    
    for k in keys:
        
        branch_list_all[k] = np.concatenate([signal_branches[k], bkg_branches[k]])
        branch_list_all[k] = branch_list_all[k].astype(np.float32)
        
    return branch_list_all"""

'def join_signal_bkg(signal_branches, bkg_branches):\n    branch_list_all = {}\n    \n    for k in keys:\n        \n        branch_list_all[k] = np.concatenate([signal_branches[k], bkg_branches[k]])\n        branch_list_all[k] = branch_list_all[k].astype(np.float32)\n        \n    return branch_list_all'

In [233]:
combined_data = join_branch(combined_data)

In [234]:
def data_before():
    ex = []
    std = []
    min = []
    max = []
    std5 = []
    
    for key in keys:
        ex.append(np.mean(combined_data[key]))
        std.append(np.std(combined_data[key]))
        min.append(np.min(combined_data[key]))
        max.append(np.max(combined_data[key]))
        std5.append(5*np.std(combined_data[key]))
        
    return ex, std, min, max, std5

In [235]:
ex, std, min, max, std5 = data_before()

for i in range(len(keys)):
    print(f'{keys[i]}: Mean: {ex[i]}, Std: {std[i]}, Min: {min[i]}, Max: {max[i]}, Std5: {std5[i]}')
    print("---------------------------------------------------------------------------------------------------")

lep_Pt_0: Mean: 81506.9453125, Std: 416091.34375, Min: 20247.578125, Max: 80278400.0, Std5: 2080456.71875
---------------------------------------------------------------------------------------------------
lep_E_0: Mean: 170083.25, Std: 2850169.5, Min: 20934.935546875, Max: 554343296.0, Std5: 14250847.5
---------------------------------------------------------------------------------------------------
lep_Eta_0: Mean: 0.004989815875887871, Std: 1.2318859100341797, Min: -2.699021577835083, Max: 2.6995906829833984, Std5: 6.159429550170898
---------------------------------------------------------------------------------------------------
lep_Phi_0: Mean: -0.0013303905725479126, Std: 1.8121583461761475, Min: -3.1413164138793945, Max: 3.141587257385254, Std5: 9.060791730880737
---------------------------------------------------------------------------------------------------
lep_EtaBE2_0: Mean: 0.003971344791352749, Std: 0.9433398246765137, Min: -2.469846248626709, Max: 2.4695277214050293, 

In [254]:
def paint(key, m, lst, color):
    fig = plt.figure()
    biny = bins(m, lst[key])
    plt.hist(lst[key], bins=biny, color=color)
    plt.title(key)
    return fig

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

pp = PdfPages('plots_before3.pdf')
for i in keys[201:]:
    fig = paint(i, 100, combined_data, "red")
    plt.savefig(pp, format='pdf')
pp.close()

In [244]:
def minus_value_filter(keys, branch):
    try:
        for key in keys:
            min = np.min(branch[key])
            max = np.max(branch[key])
            distance = abs(max - min)
            if min < 0:
                for i in range(len(branch[key])):
                    branch[key][i] = branch[key][i] + distance
        return branch
    except ValueError:
        print(key)

In [245]:
combined_data = minus_value_filter(keys, combined_data)


In [246]:
def norm(keys, branch):
    
    maxes = {}
    mins = {}
    
    for key in keys:
        max = np.max(branch[key])
        min = np.min(branch[key])
        
        
        for i in range(len(branch[key])):
            if max != 0:
                branch[key][i] = branch[key][i] / max
        if(max != 0):
            maxes[key] = max / max    
            mins[key] = min / max
        
        
    return [branch, maxes, mins]

In [247]:
from sklearn import preprocessing

def new_norm(keys, branch):
    
    for key in keys:
        branch[key] = preprocessing.normalize(branch[key].reshape(1, -1))
        branch[key] = branch[key].T
        
    return branch

In [248]:
combined_data = new_norm(keys, combined_data)

In [249]:
def bins(n, list):
    bins = []
    
    list = sorted(list)
    
    last = list[-1]
    k = np.min(list)
    while k < last:
        bins.append(k)
        k += (last - list[0])/n
    
    
    return bins

In [250]:
def cuting(list):
    mean = np.mean(list)
    std = np.std(list)
    
    filtred = []
    
    max_list = []
    min_list = []
    
    max_plus = mean + 5*std
    max_minus = mean - 5*std
    for i in range(len(list)):
        if(list[i] <= max_plus and list[i] >= max_minus):
            filtred.append(list[i])
        elif (list[i] >= max_plus):
          max_list.append(list[i])
        elif (list[i] <= max_minus):
            min_list.append(list[i])
    if (len(max_list) > 0):
        cut_max = np.min(max_list)
    else:
        cut_max = np.max(filtred)
    if(len(min_list) > 0):
        cut_min = np.max(min_list)
    else:
        cut_min = np.min(filtred)
    
    
    return [np.array(filtred), cut_max, cut_min]

In [138]:
globals_max = {}
globals_min = {}

for key in keys:
    try:
        cut = cuting(combined_data[key])
        combined_data[key] = cut[0]
        globals_max[key] = cut[1]
        globals_min[key] = cut[2]
    except ValueError:
        print(key)

In [251]:
ex, std, min, max, std5 = data_before()

for i in range(len(keys)):
    print(f'{keys[i]}: Mean: {ex[i]}, Std: {std[i]}, Min: {min[i]}, Max: {max[i]}, Std5: {std5[i]}')
    print("---------------------------------------------------------------------------------------------------")

lep_Pt_0: Mean: 0.0006981135811656713, Std: 0.0035638560075312853, Min: 0.00017342214414384216, Max: 0.687591016292572, Std5: 0.017819280037656426
---------------------------------------------------------------------------------------------------
lep_E_0: Mean: 0.00021633044525515288, Std: 0.0036251572892069817, Min: 2.6627336410456337e-05, Max: 0.7050743103027344, Std5: 0.01812578644603491
---------------------------------------------------------------------------------------------------
lep_Eta_0: Mean: 0.0035407438408583403, Std: 0.000807200965937227, Min: 0.0017689234809949994, Max: 0.005306398030370474, Std5: 0.004036004829686135
---------------------------------------------------------------------------------------------------
lep_Phi_0: Mean: 0.0034892912954092026, Std: 0.0010066184913739562, Min: 0.0017450903542339802, Max: 0.00523512065410614, Std5: 0.005033092456869781
---------------------------------------------------------------------------------------------------
lep_EtaB

In [252]:
def paint2(key, m, lst, lst2, color, color2, alpha):
    fig = plt.figure()
    biny = bins(m, lst[key])
    plt.hist(lst[key], bins=biny, color=color)
    plt.hist(lst2[key], bins=biny, color=color2, alpha=alpha)
    plt.title(key)
    return fig

In [253]:
combined_data = join_branch(combined_data)

In [218]:
signal = branch_list[-1]
background = {}
for key in keys:
    for i in range(len(signal[key])):
        signal[key][i] = combined_data[key][len(combined_data[key]) - len(signal[key]) + i]
        
    background[key] = combined_data[key][:len(combined_data[key]) - len(signal[key])]

In [ ]:
pp = PdfPages('plots_after3.pdf')
for i in keys[201:]:
    fig = paint(i, 100, combined_data, "blue")
    plt.savefig(pp, format='pdf')
pp.close()

In [27]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
pp = PdfPages('plots3.pdf')
for i in keys[201:]:
    fig = paint(i, 100, combined_data[i])
    plt.savefig(pp, format='pdf')
pp.close()

In [170]:
np.save("signal_data.npy", signal)
np.save("background_data.npy", background)
np.save("combined_data.npy", combined_data)